In [3]:
import datetime as dt
from datetime import date
import os
import sys
import numpy as np
import pandas as pd
import consultas_prazos as consultas
from consultas_prazos import prazo_viagem
from glob import glob
import sqlalchemy
import pymysql
import time, datetime
from datetime import datetime, timedelta
from datetime import timedelta, date
import time
from numpy import sort

In [85]:


########codigo para ler todos arquivos em uma pasta
arquivos = sorted(glob(r'D:\teste_visao_corr\bases\*.csv'))
all_arquivos = pd.concat((pd.read_csv(cont, sep=";") for cont in arquivos), ignore_index=True )
all_arquivos

,pedido,nf_madeira,sku,nf_fornecedor,data_expedicao_fornecedor,data_entregue,data_limite_last_mile,data_limite_cliente,data_compra,data_bip,...,transp_origem,consolidador,origem_y,transportadora_nota_y,prazo_viagem,partition_0,flag_atraso_fornecedor,flag_atraso_transporte,flag_atraso_coleta,flag_atraso_redespacho
0,Z27956049,46,699092,5709.0,2022-09-22,2022-09-21,2022-10-07,2022-10-07,2022-09-17,2022-09-19,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0
1,Z27359413,2473455,729312,800136.0,2022-08-31,2022-09-02,2022-09-08,2022-09-08,2022-08-20,2022-08-30,...,RIO - BULKYLOGCD JDI Fulfillment,CD Fulfillment,CD JDI Fulfillment,RIO - BULKYLOG,2.0,2022-10-27,0.0,0.0,0.0,NaN
2,Z27858888,2481819,202279,NaN,2022-09-19,2022-09-28,2022-10-19,2022-10-26,2022-09-13,NaN,...,RECIFE - BULKYCD JDI Fulfillment,CD Fulfillment,CD JDI Fulfillment,RECIFE - BULKY,5.0,2022-10-27,NaN,0.0,NaN,NaN
3,Z27329117,2468412,537926,NaN,2022-08-26,2022-09-07,2022-09-15,2022-09-14,2022-08-18,NaN,...,DOMINALOG - SPCD JDI Fulfillment,CD Fulfillment,CD JDI Fulfillment,DOMINALOG - SP,0.0,2022-10-27,NaN,0.0,NaN,NaN
4,Z27130179,2470340,312103,585648.0,2022-08-25,2022-09-01,2022-09-12,2022-09-16,2022-08-10,2022-08-23,...,FCB BRASILCD JDI Fulfillment,CD Fulfillment,CD JDI Fulfillment,FCB BRASIL,0.0,2022-10-27,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405761,Z28610442,7080501,696508,142907.0,2022-10-21,2022-10-29,2022-11-01,2022-10-31,2022-10-19,2022-10-20,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN
405762,Z28606962,7083351,487962,833678.0,2022-10-24,2022-10-28,2022-11-09,2022-11-01,2022-10-19,2022-10-21,...,CAJAMAR - BULKYDS SC - Oeste G,Cons. Pinhalzinho,DS SC - Oeste G,CAJAMAR - BULKY,2.0,2022-10-27,0.0,0.0,1.0,NaN
405763,Z28629157,7083967,709467,834008.0,2022-10-24,2022-10-28,2022-11-04,2022-11-03,2022-10-20,2022-10-21,...,ARUJA - BULKYDS SC - Oeste G,Cons. Pinhalzinho,DS SC - Oeste G,ARUJA - BULKY,2.0,2022-10-27,0.0,0.0,1.0,NaN
405764,Z28367159,2614978,243647,1749524.0,2022-10-19,2022-10-24,2022-10-28,2022-10-31,2022-10-08,2022-10-17,...,PINHAIS - BULKYDS SC - Leste Leve,Cons. Pinhais,DS SC - Leste Leve,PINHAIS - BULKY,0.0,2022-10-27,0.0,0.0,1.0,NaN


In [89]:
all_arquivos['data_entregue'] =  all_arquivos['data_entregue'].astype('datetime64')
dale = all_arquivos.groupby(pd.Grouper(key='data_entregue', freq='w')).sum().iloc[1:]

dale.describe()

,nf_madeira,sku,nf_fornecedor,prazo_interno_redes,prazo_redes,prazo_adicional_redes,prazo_interno_diretas,prazo_diretas,prazo_adicional_diretas,prazo_viagem,flag_atraso_fornecedor,flag_atraso_transporte,flag_atraso_coleta,flag_atraso_redespacho
count,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000,8.000000,8.0,8.0,8.0,8.000000,8.000000,8.000000,8.000000,8.000000
mean,3.023845e+11,2.651871e+10,1.949270e+10,1477.625000,7441.625000,512.250000,0.0,0.0,0.0,46987.625000,1453.000000,2280.625000,27695.250000,4423.500000
std,2.333008e+10,2.037857e+09,1.463058e+09,674.304072,4846.206644,202.016796,0.0,0.0,0.0,3268.008871,195.367785,383.435574,2330.296716,1435.122195
min,2.608271e+11,2.274732e+10,1.695347e+10,471.000000,1717.000000,315.000000,0.0,0.0,0.0,42244.000000,1198.000000,1802.000000,24245.000000,2888.000000
25%,2.909512e+11,2.541510e+10,1.854927e+10,1089.000000,2653.250000,386.250000,0.0,0.0,0.0,45311.750000,1329.000000,1935.750000,26699.250000,3247.250000
50%,3.041984e+11,2.679352e+10,1.976742e+10,1579.000000,7927.500000,449.000000,0.0,0.0,0.0,46872.500000,1454.500000,2320.500000,28001.500000,4126.500000
75%,3.208841e+11,2.766410e+10,2.032272e+10,1844.250000,10737.250000,579.500000,0.0,0.0,0.0,47627.250000,1574.500000,2642.750000,28923.250000,5267.750000
max,3.305702e+11,2.940262e+10,2.163599e+10,2513.000000,15019.000000,908.000000,0.0,0.0,0.0,53400.000000,1713.000000,2698.000000,30864.000000,6683.000000
